In [13]:
from angle_emb import AnglE
from angle_emb.utils import cosine_similarity

#Funktioniert
#https://github.com/SeanLee97/AnglE
#Without Prompts: no need to specify a prompt. Just input a list of strings or a single string.

# Initialisiere das AnglE-Modell mit UAE-Large-V1
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls')

# Liste der Texte
texts = [
    'Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.',
    'Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.',
    'Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume'
]

# Texte mit dem Modell codieren
doc_vecs = angle.encode(texts)

# Vergleich der Vektoren und Ausgabe der zugehörigen Texte mit ihrer Kosinus-Ähnlichkeit
for i, dv1 in enumerate(doc_vecs):
    for j, dv2 in enumerate(doc_vecs[i+1:], start=i+1):
        similarity = cosine_similarity(dv1, dv2)
        print(f"Vergleich zwischen Text {i+1} und Text {j+1}:")
        print(f"Text {i+1}: {texts[i]}")
        print(f"Text {j+1}: {texts[j]}")
        print(f"Kosinus-Ähnlichkeit: {similarity.item()}\n")

Vergleich zwischen Text 1 und Text 2:
Text 1: Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.
Text 2: Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.
Kosinus-Ähnlichkeit: 0.9658278292827439

Vergleich zwischen Text 1 und Text 3:
Text 1: Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.
Text 3: Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume
Kosinus-Ähnlichkeit: 0.9342922266833731

Vergleich zwischen Text 2 und Text 3:
Text 2: Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.
Text 3: Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume
Kosinus-Ähnlichkeit: 0.9004386113439647



In [31]:
from angle_emb import AnglE, Prompts
from angle_emb.utils import cosine_similarity

# Initialisiere das AnglE-Modell
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls')

#funktioniert
#https://github.com/SeanLee97/AnglE
#With Prompts


# Liste der Dokumente als einfache Strings
texts = [
    'Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.',
    'Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.',
    'Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume.'
]

# Dokumente codieren mit Prompt A
doc_vecs = angle.encode([{'text': t} for t in texts], to_numpy=True, prompt=Prompts.A)

# Vergleich der Anfrage mit jedem Dokument
for i, dv1 in enumerate(doc_vecs):
    for j, dv2 in enumerate(doc_vecs[i+1:], start=i+1):
        similarity = cosine_similarity(dv1, dv2)
        print(f"Vergleich zwischen Text {i+1} und Text {j+1}:")
        print(f"Text {i+1}: {texts[i]}")
        print(f"Text {j+1}: {texts[j]}")
        print(f"Kosinus-Ähnlichkeit: {similarity.item()}\n")
        
        Prompts.list_prompts()

Vergleich zwischen Text 1 und Text 2:
Text 1: Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.
Text 2: Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.
Kosinus-Ähnlichkeit: 0.9787534623563109

Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'
Vergleich zwischen Text 1 und Text 3:
Text 1: Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.
Text 3: Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume.
Kosinus-Ähnlichkeit: 0.9578784755546029

Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'
Vergleich zwischen Text 2 und Text 3:
Text 2

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from angle_emb import AnglE, Prompts
from angle_emb.utils import cosine_similarity

#funktioniert
#https://github.com/SeanLee97/AnglE
#Infer LLM-based Models

# 1. Lade das Hauptmodell und die LoRA-Gewichte auf der CPU
model = AutoModelForCausalLM.from_pretrained('NousResearch/Llama-2-7b-hf', torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-hf')

# Lade das LoRA-Modell auf der CPU
lora_model = PeftModel.from_pretrained(model, 'SeanLee97/angle-llama-7b-nli-v2', torch_dtype=torch.float16)

# 2. Initialisiere AnglE mit dem Modell und den LoRA-Gewichten (ohne CUDA)
angle = AnglE.from_pretrained(
    'NousResearch/Llama-2-7b-hf',
    pretrained_lora_path='SeanLee97/angle-llama-7b-nli-v2',
    pooling_strategy='last',
    is_llm=True,
    torch_dtype=torch.float16
)

# 3. Vektoren für die Texte erzeugen (mit dem passenden Prompt)
#texts = [
#    {'text': 'The sky is blue, and the sun is shining bright. The wind gently blows through the trees.'},
#    {'text': 'The sky is not blue, and the sun is not shining bright. The wind is not gently blowing through the trees.'},
#    {'text': 'The sky appears blue, and the sun shines strongly. A gentle wind moves the trees.'},
#    {'text': 'The dog is playing with the ball.'}
#]

texts = [
    {'text': 'Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.'},
    {'text': 'Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.'},
    {'text': 'Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume'},
]

doc_vecs = angle.encode(texts, prompt=Prompts.A)

# 4. Berechne die Kosinus-Ähnlichkeiten zwischen den Texten
for i, dv1 in enumerate(doc_vecs):
    for j, dv2 in enumerate(doc_vecs[i+1:], start=i+1):
        similarity = cosine_similarity(dv1, dv2)
        print(f'Similarity between text {i+1} and text {j+1}: {similarity.item()}')

# 5. Liste alle vordefinierten Prompts auf
print('All predefined prompts:', Prompts.list_prompts())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

AttributeError: 'LlamaForCausalLM' object has no attribute 'encode'

In [2]:
import os
import torch
from angle_emb import AnglE, Prompts
from angle_emb.utils import cosine_similarity

#funktioniert
#https://github.com/SeanLee97/AnglE
#Infer BiLLM-Model

# Setze die Umgebung für BiLLM
os.environ['BiLLM_START_INDEX'] = '31'

# 1. Initialisiere AnglE mit BiLLM und dem Modell sowie den LoRA-Gewichten (ohne CUDA)
angle = AnglE.from_pretrained(
    'NousResearch/Llama-2-7b-hf',
    pretrained_lora_path='SeanLee97/bellm-llama-7b-nli',  # Angepasst auf das BiLLM Modell
    pooling_strategy='last',
    is_llm=True,
    apply_billm=True,
    billm_model_class='LlamaForCausalLM',
    torch_dtype=torch.float16
)

# 2. Vektoren für die Texte erzeugen (mit dem passenden Prompt)
texts = [
    {'text': 'Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.'},
    {'text': 'Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.'},
    {'text': 'Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume'},
]



# Nutze den BiLLM-Prompt, um die Vektoren zu erzeugen
doc_vecs = angle.encode(texts, prompt='The representative word for sentence {text} is:"')

# 3. Berechne die Kosinus-Ähnlichkeiten zwischen den Texten
for i, dv1 in enumerate(doc_vecs):
    for j, dv2 in enumerate(doc_vecs[i+1:], start=i+1):
        similarity = cosine_similarity(dv1, dv2)
        print(f'Similarity between text {i+1} and text {j+1}: {similarity.item()}')

# 4. Liste alle vordefinierten Prompts auf
print('All predefined prompts:', Prompts.list_prompts())

INFO:AnglE:LLM detected, automatically set apply_lora=True.If it is wrong, you can manually set `apply_lora`.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
INFO:BiLLM:Here is the Bi-LlamaModel! BiLLM_START_INDEX=31


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:AnglE:Load lora weight from SeanLee97/bellm-llama-7b-nli


adapter_config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

C:\Users\Finn\Work\Bachelor\Bachelor\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Finn\.cache\huggingface\hub\models--SeanLee97--bellm-llama-7b-nli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.45 `position_ids` will be removed and `position_embeddings` will be mandatory.


Similarity between text 1 and text 2: 0.797877674990915
Similarity between text 1 and text 3: 0.95167154252485
Similarity between text 2 and text 3: 0.769807873441355
Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'
All predefined prompts: None


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from angle_emb import AnglE, Prompts
from angle_emb.utils import cosine_similarity

# 1. Lade das Hauptmodell und die LoRA-Gewichte auf der CPU
model = AutoModelForCausalLM.from_pretrained('NousResearch/Llama-2-7b-hf', torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-hf')

# Lade das LoRA-Modell auf der CPU
lora_model = PeftModel.from_pretrained(model, 'SeanLee97/angle-llama-7b-nli-v2', torch_dtype=torch.float16)

# 2. Initialisiere AnglE mit dem Modell und den LoRA-Gewichten (ohne CUDA)
angle = AnglE.from_pretrained(
    'NousResearch/Llama-2-7b-hf',
    pretrained_lora_path='SeanLee97/angle-llama-7b-nli-v2',
    pooling_strategy='last',
    is_llm=True,
    torch_dtype=torch.float16
)

# 3. Vektoren für die Texte erzeugen (mit dem passenden Prompt)
#texts = [
#    {'text': 'The sky is blue, and the sun is shining bright. The wind gently blows through the trees.'},
#    {'text': 'The sky is not blue, and the sun is not shining bright. The wind is not gently blowing through the trees.'},
#    {'text': 'The sky appears blue, and the sun shines strongly. A gentle wind moves the trees.'},
#    {'text': 'The dog is playing with the ball.'}
#]

texts = [
    {'text': 'Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.'},
    {'text': 'Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.'},
    {'text': 'Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume'},
]

doc_vecs = angle.encode(texts, prompt=Prompts.A)

# 4. Berechne die Kosinus-Ähnlichkeiten zwischen den Texten
for i, dv1 in enumerate(doc_vecs):
    for j, dv2 in enumerate(doc_vecs[i+1:], start=i+1):
        similarity = cosine_similarity(dv1, dv2)
        print(f'Similarity between text {i+1} and text {j+1}: {similarity.item()}')

# 5. Liste alle vordefinierten Prompts auf
print('All predefined prompts:', Prompts.list_prompts())

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\Finn\Work\Bachelor\Bachelor\venv\Lib\site-packages\peft\utils\save_and_load.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(file

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:AnglE:Load lora weight from SeanLee97/angle-llama-7b-nli-v2


Similarity between text 1 and text 2: 0.6794261320640602
Similarity between text 1 and text 3: 0.9498970711361487
Similarity between text 2 and text 3: 0.7047534430896022
Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'
All predefined prompts: None


In [3]:
import torch
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F

# Festlegen der Dimension für die Embeddings
matryoshka_dim = 1024  # Optionen: 64, 128, 256, 512, 768, 1024

# Laden des Modells
model = SentenceTransformer("aari1995/German_Semantic_V3", trust_remote_code=True, truncate_dim=matryoshka_dim)

# Sätze im gegebenen Format
texts = [
    {'text': 'Der Himmel ist blau und die Sonne scheint hell. Der Wind weht sanft durch die Bäume.'},
    {'text': 'Der Himmel ist nicht blau und die Sonne scheint nicht hell. Der Wind weht nicht sanft durch die Bäume.'},
    {'text': 'Der Himmel zeigt sich in Blau, und die Sonne strahlt stark. Ein sanfter Wind bewegt die Bäume'},
]

# Extrahiere die Sätze aus dem Wörterbuch
sentences = [item['text'] for item in texts]

# Berechnen der Embeddings (Verwenden von FP16 für weniger Speicherverbrauch)
embeddings = model.encode(sentences, convert_to_tensor=True).half()

# Berechne die Kosinus-Ähnlichkeit zwischen den Satz-Embeddings
similarities = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)

# Ausgabe der Kosinus-Ähnlichkeit im gewünschten Format
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        print(f'Similarity between text {i+1} and text {j+1}: {similarities[i, j].item():.4f}')

Similarity between text 1 and text 2: 0.4841
Similarity between text 1 and text 3: 0.9434
Similarity between text 2 and text 3: 0.4651


In [4]:
import torch
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F

# Festlegen der Dimension für die Embeddings
matryoshka_dim = 1024  # Optionen: 64, 128, 256, 512, 768, 1024

# Laden des Modells
model = SentenceTransformer("aari1995/German_Semantic_V3", trust_remote_code=True, truncate_dim=matryoshka_dim)

# Sätze im gegebenen Format
texts = [
{'text': 'Der Himmel ist heute von einem strahlenden, tiefen Blau, das sich in den unendlichen Weiten des Himmels erstreckt und die ganze Atmosphäre in ein beruhigendes Licht taucht. Die Sonne steht hoch am Himmel und scheint mit großer Intensität, ihre Strahlen brechen sich in der Luft und erzeugen funkelnde Reflexionen auf jeder Oberfläche, die sie berührt. Der Wind, der nur schwach weht, gleitet sanft durch die Blätter der Bäume, lässt sie leise rascheln und schwingt in einem zarten Rhythmus, der die Stille des Tages auf angenehme Weise untermalt. Die Natur scheint in völliger Harmonie, als ob der Himmel, die Sonne und der Wind gemeinsam eine symphonische Ruhe ausstrahlen, die den Tag vollkommen erscheinen lässt.'},

{'text': 'Der Himmel ist heute keineswegs von einem strahlenden Blau erfüllt, sondern erscheint eher düster und grau, ohne jeglichen Schimmer von Licht, der die Stimmung erhellen könnte. Die Sonne, die normalerweise am Himmel hell und warm leuchtet, ist vollständig verdeckt und wirft keinen einzigen Strahl auf die Erde, wodurch alles unter einem bleiernen, schweren Schatten liegt. Der Wind, der sonst sanft und ruhig durch die Bäume weht, regt sich kaum und erzeugt kein Rascheln in den Blättern, sodass die Umgebung still und drückend wirkt. Nichts an diesem Tag strahlt Harmonie oder Ruhe aus, denn der Himmel, die Sonne und der Wind sind alle in eine fast unheimliche Stille und Dunkelheit gehüllt, die den Tag träge und bedrückend erscheinen lässt.'},

{'text': 'Der Himmel erstrahlt heute in einem leuchtenden und intensiven Blau, das sich hoch über uns erstreckt und den Himmel in einer faszinierenden Tiefe erscheinen lässt. Die goldene Sonne, die ihre wärmenden Strahlen unermüdlich auf die Erde schickt, thront majestätisch am Himmel und erfüllt die Luft mit einem leichten, flimmernden Glanz, der jede Landschaft in ein sanftes, warmes Licht taucht. Ein milder Windhauch zieht durch die Kronen der Bäume und bringt die Blätter dazu, ein leises, beruhigendes Geräusch von sich zu geben, das sich mit den anderen sanften Geräuschen der Natur zu einem harmonischen Klangteppich vereint. Alles an diesem Tag wirkt in vollkommener Balance, als ob der Himmel, die Sonne und der Wind ein sorgfältig abgestimmtes Zusammenspiel eingehen, um einen Moment der vollkommenen Ruhe und Schönheit zu erschaffen.'}
]

# Extrahiere die Sätze aus dem Wörterbuch
sentences = [item['text'] for item in texts]

# Berechnen der Embeddings (Verwenden von FP16 für weniger Speicherverbrauch)
embeddings = model.encode(sentences, convert_to_tensor=True).half()

# Berechne die Kosinus-Ähnlichkeit zwischen den Satz-Embeddings
similarities = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)

# Ausgabe der Kosinus-Ähnlichkeit im gewünschten Format
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        print(f'Similarity between text {i+1} and text {j+1}: {similarities[i, j].item():.4f}')

Similarity between text 1 and text 2: 0.6338
Similarity between text 1 and text 3: 0.9565
Similarity between text 2 and text 3: 0.5654
